# Annotate via LangGraph

In [ ]:
import { readFileSync } from 'node:fs';
import { START, END, StateGraph } from "@langchain/langgraph";
import { Annotations } from "@common/annotations.ts";

import { EXPERIMENTS_DIR, SERVER_DATA_DIR } from '../server/src/util/fileUtils.ts';
import { getNotebookLogger } from '../server/src/Logger.ts';
import { StateInfo, responseContent } from '../server/src/agents/agent.ts';
import { encodeAnnotationsNode } from "../server/src/agents/nodes/encodeAnnotationsNode.ts";
import { modelNode } from "../server/src/agents/nodes/modelNode.ts";
import { HumanMessage, SystemMessage } from "@langchain/core/messages";

export const annNode = (state: typeof StateInfo.State) => {
  const PROMPT = readFileSync(`${EXPERIMENTS_DIR}/annotateNodePromptCategories.txt`, 'utf-8');
  state.logger.debug(`System Message:\n${PROMPT}`);
  state.logger.debug(`Human Message:\n${state.systemData}`);
  return { messages: [
    new SystemMessage(PROMPT),
    new HumanMessage(state.systemData)
  ]};
}

// Define a new graph
const workflow = new StateGraph(StateInfo)
  .addNode("encodeAnnotations", encodeAnnotationsNode)
  .addNode("annotate", annNode)
  .addNode("model", modelNode)
  .addEdge(START, "encodeAnnotations")
  .addEdge("encodeAnnotations","annotate")
  .addEdge("annotate", "model")
  .addEdge("model", END);

// Compile graph
export const graph = workflow.compile();

const logger = getNotebookLogger();
const userUUID: string = "0";
const lhsText = readFileSync(`${SERVER_DATA_DIR}/AES/selected-text.txt`, 'utf-8');
const rhsText = readFileSync(`${SERVER_DATA_DIR}/AES/pre-written.txt`, 'utf-8');
const currentAnnotations: Annotations = { "mappings": [], "lhsLabels": [], "rhsLabels": [] };

const config = { configurable: { thread_id: userUUID } };
const output = await graph.invoke({ lhsText: lhsText, rhsText: rhsText, currentAnnotations: currentAnnotations, logger: logger }, config);
logger.info(responseContent(output));


Sending messages to LLM.
Received response from LLM.
LLM usage: 19242 input tokens, 2277 output tokens
### CHAIN-OF-THOUGHT ANALYSIS

I'll analyze the LHS TEXT and identify semantically contiguous segments, labeling each with the appropriate semantic category.

1. The first segment introduces the AES algorithm and its variants, explaining the general structure and key differences. This is an overview section that provides context.

2. The next segment describes the CIPHER() function and its components, including the four transformations (SubBytes, ShiftRows, MixColumns, AddRoundKey).

3. The pseudocode for CIPHER() is presented, followed by explanations of each transformation in detail.

4. Each transformation (SubBytes, ShiftRows, MixColumns, AddRoundKey) is described in its own subsection with mathematical definitions.

5. The inverse cipher operations are described in their own subsections.

6. The document also includes tables, figures, and algorithms in pseudocode format.

Let me 